In [1]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")


In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="codebasics_faqs.csv",source_column='prompt')

data = loader.load()


In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

# Initialize Google Gemini embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")



# Store embeddings in ChromaDB with persistent storage
vector_db = Chroma.from_documents(
    documents=data, embedding=embeddings, persist_directory="chroma_db"
)

print("ChromaDB vector database saved successfully.")  # ✅ No persist() needed


ChromaDB vector database saved successfully.


In [7]:
# Load ChromaDB from disk
vector_db = Chroma(persist_directory="chroma_db", embedding_function=embeddings)

# Query ChromaDB
query = "how about job placememt support?"
docs = vector_db.similarity_search(query, k=1)

retriever = vector_db.as_retriever(search_kwargs={"k": 2})


In [8]:
rdocs = retriever.get_relevant_documents("how about job placememt support?")

C:\Users\shaik\AppData\Local\Temp\ipykernel_3056\2837020523.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rdocs = retriever.get_relevant_documents("how about job placememt support?")


In [9]:
rdocs

[Document(metadata={'row': 11, 'source': 'Do you provide any job assistance?'}, page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.'),
 Document(metadata={'row': 33, 'source': 'Will this course guarantee me a job?'}, page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless pr

In [15]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question} 

"""

PROMPT = PromptTemplate(
    template=prompt_template,input_variables=["context","question"]
)


In [16]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type (llm=llm,
                                    chain_type="stuff",
                                    retriever=retriever,
                                    input_key="query",
                                    return_source_documents=True,
                                    chain_type_kwargs={"prompt":PROMPT}
                                    )


In [20]:
chain("I've a mac computer, can i use powerbi on it?")

{'query': "I've a mac computer, can i use powerbi on it?",
 'result': 'Hi\n\nYou can use VirtualBox to create a virtual machine and install Windows on it. This will allow you to run Power BI and Excel on your Mac.\n\nIf you\'re not familiar with setting up a virtual machine, there are many resources available on YouTube that can guide you through the process. Simply search for "installing virtual machines" and you\'ll find plenty of helpful videos.',
 'source_documents': [Document(metadata={'row': 31, 'source': 'Does Power BI work in Mac OS/Ubuntu?'}, page_content='prompt: Does Power BI work in Mac OS/Ubuntu?\nresponse: Power BI desktop works only in Windows OS. Please look into the system requirements section on this page. However, you can use a virtual machine to install and work with Power BI in other Operating systems.'),
  Document(metadata={'row': 44, 'source': 'How can I use PowerBI on my Mac system?'}, page_content='prompt: How can I use PowerBI on my Mac system?\nresponse: Hi\